In [3]:
##Arxiv Research
#Tool creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

  

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=350)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [5]:
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

'arxiv'

In [6]:
tools=[wiki,arxiv]

In [11]:
#Custom Tools[RAG tool]
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings,HuggingFaceEmbeddings

embeddings=OpenAIEmbeddings()
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
fin_split=text_splitter.split_documents(docs)
db=FAISS.from_documents(fin_split,embeddings)
retriever=db.as_retriever()






In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search any information about langsmith")
retriever_tool.name

'langsmith_search'

In [27]:
#Combine the tools with Agents and LLM's

import openai
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(model_name="gemma2-9b-it",groq_api_key=groq_api_key)


In [28]:
## Prompt Template

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
tools=[wiki,arxiv,retriever_tool]

In [30]:
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)


agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [31]:
from langchain.agents import AgentExecutor

age_exec=AgentExecutor(agent=agent,tools=tools,verbose=True)

age_exec

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [35]:
age_exec.invoke({"input":"Attention is all you need"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention is all you need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundationa
Invoking: `arxiv` with `{'query': 'transformer model'}`


Published: 2012-10-23
Title: Model Validation in Ontology Based Transformations
Authors: Jesús M. Almendros-Jiménez, Luis Iribarne
Summary: Model Driven Engineering (MDE) is an emerging approach of software
engineering. MDE emphasizes the construction of models from which the
implementation should bIt seems like the provided information is not directly related to the transformer model.  Could you clarify your query?  Perhaps you meant to ask about a specific aspect of transfo

{'input': 'Attention is all you need',
 'output': 'It seems like the provided information is not directly related to the transformer model.  Could you clarify your query?  Perhaps you meant to ask about a specific aspect of transformer models, or a different type of model? \n\n\n'}